In [1]:
import pandas as pd
import json


In [2]:
with open('proj5_params.json', encoding="utf-8") as json_file:
    params = json.load(json_file)

In [3]:
#TASK 1
df = pd.read_csv('proj5_timeseries.csv')
print(df.shape)
print(df.columns)
print(params["original_frequency"])
#df.iloc[[2051, 2052, 2999]]

(4385, 5)
Index(['Date', 'Consumption [Wh]', 'Wind', 'Solar', 'Wind+Solar'], dtype='object')
D


In [4]:
new_names = []
for col in df.columns:
    new_name=""
    for letter in col:
        ascii_code = ord(letter)
        if ascii_code in range(65, 91) or ascii_code in range(97, 123) or ascii_code in range(48, 58):
            new_name += letter.lower()
        else:
            new_name+="_"
    new_names.append(new_name)

df.columns = new_names

df[df.columns[0]] = pd.to_datetime(df[df.columns[0]], format='mixed')
df.set_index(df.columns[0], inplace=True)
dfn = df.asfreq(params["original_frequency"])
dfn.to_pickle("proj5_ex01.pkl")
dfn.iloc[[2051, 2052, 2999]]


,consumption__wh_,wind,solar,wind_solar
date,,,,
2011-08-14,979.297,71.681,NaN,NaN
2011-08-15,1218.898,44.782,NaN,NaN
2014-03-19,1524.762,405.654,66.495,472.149


In [5]:
dfn.head()

,consumption__wh_,wind,solar,wind_solar
date,,,,
2006-01-01,1069.184,NaN,NaN,NaN
2006-01-02,1380.521,NaN,NaN,NaN
2006-01-03,1442.533,NaN,NaN,NaN
2006-01-04,1457.217,NaN,NaN,NaN
2006-01-05,1477.131,NaN,NaN,NaN


In [6]:
#TASK 2
df_target_freq = dfn.asfreq(params["target_frequency"])
df_target_freq.to_pickle("proj5_ex02.pkl")
df_target_freq.head()

,consumption__wh_,wind,solar,wind_solar
date,,,,
2006-01-01,1069.184,NaN,NaN,NaN
2006-01-08,1207.985,NaN,NaN,NaN
2006-01-15,1265.475,NaN,NaN,NaN
2006-01-22,1248.111,NaN,NaN,NaN
2006-01-29,1252.215,NaN,NaN,NaN


In [7]:
#TASK 3
how = f"{params['downsample_periods']}{params['downsample_units']}"
print(how)
df_downsampled = dfn.resample(how).sum(min_count=int(params["downsample_periods"]))
df_downsampled.to_pickle("proj5_ex03.pkl")
df_downsampled

3d


,consumption__wh_,wind,solar,wind_solar
date,,,,
2006-01-01,3892.23800,NaN,NaN,NaN
2006-01-04,4337.77500,NaN,NaN,NaN
2006-01-07,4037.59500,NaN,NaN,NaN
2006-01-10,4707.36700,NaN,NaN,NaN
2006-01-13,4170.42200,NaN,NaN,NaN
...,...,...,...,...
2017-12-17,4462.39225,484.667,37.310,521.977
2017-12-20,4503.29557,578.692,26.177,604.869
2017-12-23,3525.21153,2148.306,34.164,2182.470


In [8]:
#TASK 4

how = f"{params['upsample_periods']}{params['upsample_units']}"
print(how)
df_upsampled = dfn.resample(how)

df_upsampled = df_upsampled.interpolate(
        method=params['interpolation'],
        order=params['interpolation_order']
    )

original_timedelta = pd.Timedelta(f'1{params["original_frequency"]}')
print(original_timedelta)
new_timedelta = pd.Timedelta(how)
print(new_timedelta)
scaling_factor =  new_timedelta/original_timedelta 
print(scaling_factor)

df_upsampled = df_upsampled * scaling_factor
df_upsampled.to_pickle('proj5_ex04.pkl')
df_upsampled

2h
1 days 00:00:00
0 days 02:00:00
0.08333333333333333


,consumption__wh_,wind,solar,wind_solar
date,,,,
2006-01-01 00:00:00,89.098667,NaN,NaN,NaN
2006-01-01 02:00:00,92.515716,NaN,NaN,NaN
2006-01-01 04:00:00,95.668110,NaN,NaN,NaN
2006-01-01 06:00:00,98.566792,NaN,NaN,NaN
2006-01-01 08:00:00,101.222704,NaN,NaN,NaN
...,...,...,...,...
2017-12-30 16:00:00,95.117424,63.173803,0.423908,63.597711
2017-12-30 18:00:00,94.371167,62.769170,0.626352,63.395522
2017-12-30 20:00:00,93.643499,62.132939,0.897118,63.030057


In [9]:
print(df_upsampled.iloc[[0,1,2,3,4,5,6,7,8,9,10,11],0].sum())
print(df_upsampled.iloc[[12,13,14,15,16,17,18,19,20,21,22,23],0].sum())

1241.9017679938975
1420.1871197375838


In [10]:
#TASK 5
df_sensor = pd.read_pickle('proj5_sensors.pkl')
df_sensor

,device_id,value
timestamp,,
2022-11-25 00:00:25,25,6.693750
2022-11-25 00:01:25,25,6.725000
2022-11-25 00:02:25,25,6.756250
2022-11-25 00:03:25,25,6.787500
2022-11-25 00:04:25,25,6.818750
...,...,...
2022-11-27 23:55:29,47,6.022222
2022-11-27 23:56:29,47,6.016667
2022-11-27 23:57:29,47,6.011111


In [11]:
# df_sensor = df_sensor.reset_index()
# df_sensor

In [12]:
#dfp_sensor = df_sensor.pivot(index='timestamp', columns='device_id', values='value')
dfp_sensor = df_sensor.pivot(columns='device_id', values='value')
dfp_sensor

device_id,25,26,27,28,29,30,31,32,33,34,...,38,39,40,41,42,43,44,45,46,47
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-11-25 00:00:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:04,NaN,NaN,NaN,5.6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,7.7,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-27 23:59:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.133333,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-27 23:59:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.584211,NaN,NaN,NaN,NaN
2022-11-27 23:59:51,NaN,NaN,NaN,NaN,NaN,6.094737,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
freq = f"{params['sensors_periods']}{params['sensors_units']}"
print(freq)
new_index = pd.date_range(dfp_sensor.index.round(freq).min(), dfp_sensor.index.round(freq).max(), freq=freq)
#dfr_sensor = dfp_sensor.resample(freq).mean().interpolate(method='linear')
#dfp_sensor.reindex(new_index)

10s


In [14]:
dfp2 = dfp_sensor.reindex(new_index.union(dfp_sensor.index)).interpolate(method="linear")
dfp2

device_id,25,26,27,28,29,30,31,32,33,34,...,38,39,40,41,42,43,44,45,46,47
2022-11-25 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:04,NaN,NaN,NaN,5.600000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:08,NaN,NaN,NaN,5.599533,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:09,NaN,NaN,NaN,5.599066,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-27 23:59:50,4.305263,4.482353,5.296078,4.721053,6.380952,6.094932,5.155455,5.190476,4.24,5.888235,...,5.6,4.515789,2.3,5.133333,5.3,5.584211,4.7,6.2,4.005882,6.0
2022-11-27 23:59:51,4.305263,4.482353,5.298039,4.721053,6.380952,6.094737,5.155455,5.190476,4.24,5.888235,...,5.6,4.515789,2.3,5.133333,5.3,5.584211,4.7,6.2,4.005882,6.0
2022-11-27 23:59:55,4.305263,4.482353,5.300000,4.721053,6.380952,6.094737,5.155455,5.190476,4.24,5.888235,...,5.6,4.515789,2.3,5.133333,5.3,5.584211,4.7,6.2,4.005882,6.0
2022-11-27 23:59:59,4.305263,4.482353,5.300000,4.721053,6.380952,6.094737,5.155455,5.190476,4.24,5.888235,...,5.6,4.515789,2.3,5.133333,5.3,5.584211,4.7,6.2,4.005882,6.0


In [15]:
dfp3 = dfp2.reindex(new_index)
dfp3.head()

device_id,25,26,27,28,29,30,31,32,33,34,...,38,39,40,41,42,43,44,45,46,47
2022-11-25 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:10,NaN,NaN,NaN,5.598599,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:20,NaN,NaN,NaN,5.596731,6.737542,NaN,NaN,NaN,NaN,NaN,...,NaN,6.5,3.237500,NaN,7.702614,NaN,NaN,NaN,6.798051,NaN
2022-11-25 00:00:30,6.698380,NaN,NaN,5.593462,6.745791,NaN,5.438030,NaN,NaN,5.367901,...,7.070370,6.5,3.240741,NaN,7.708715,NaN,NaN,7.101559,6.791228,7.727778
2022-11-25 00:00:40,6.701852,5.085926,NaN,5.592061,6.749327,NaN,5.437879,7.545267,NaN,5.358642,...,7.072222,6.5,3.242130,NaN,7.711329,NaN,NaN,7.103899,6.788304,7.729293


In [16]:

complete_cases = dfp3.dropna()
complete_cases.to_pickle('proj5_ex05.pkl')
complete_cases.head()

device_id,25,26,27,28,29,30,31,32,33,34,...,38,39,40,41,42,43,44,45,46,47
2022-11-25 00:01:00,6.711111,5.109630,5.192130,5.588325,6.758754,5.562434,5.437475,7.551852,6.749673,5.333951,...,7.077160,6.5,3.245833,7.106173,7.718301,5.215595,7.022896,7.110136,6.780507,7.733333
2022-11-25 00:01:10,6.716898,5.124444,5.203704,5.585990,6.764646,5.571252,5.437222,7.555967,6.751852,5.318519,...,7.080247,6.5,3.248148,7.112346,7.722658,5.235088,7.026263,7.114035,6.775634,7.735859
2022-11-25 00:01:20,6.721528,5.136296,5.212963,5.584122,6.769360,5.578307,5.437020,7.559259,6.753595,5.306173,...,7.082716,6.5,3.250000,7.117284,7.726144,5.250682,7.028956,7.117154,6.771735,7.737879
2022-11-25 00:01:30,6.729630,5.157037,5.229167,5.580853,6.777609,5.590653,5.436667,7.565021,6.756645,5.290741,...,7.087037,6.5,3.253241,7.125926,7.732244,5.277973,7.033670,7.122612,6.764912,7.741414
2022-11-25 00:01:40,6.733102,5.165926,5.236111,5.579452,6.781145,5.595944,5.436515,7.567490,6.757952,5.285185,...,7.088889,6.5,3.254630,7.129630,7.734858,5.289669,7.035690,7.124951,6.761988,7.742929
